In [ ]:
import numpy as np
import pandas as pd
import json
import time
import cProfile
import os
import copy
from deepdiff import DeepDiff
from timeit import default_timer as timer
import matplotlib.pyplot as plt

In [2]:
from market.market_state_v1 import MarketStateAttribute
# import Parser
from environment.parser_new import SnapshotParser, MessagePacketParser

ModuleNotFoundError: No module named 'environment.parser_new'

In [3]:
path = "/Users/florianewald/PycharmProjects/Level3-Data-Analysis/sample_msg_data/"

In [4]:
# TODO: test market state with more data!

In [5]:
file = "DE0005190003.XETR_20220202T080014_20220202T120000"
#file = "DE0005190003.XETR_20220201T080007_20220201T120000"

In [6]:
def load_data(path):
    start = timer()
    # load snapshot start
    file = open(f"{path}/snapshot_start.json")
    snapshot_start = json.load(file)
    
    # load snapshot end
    file = open(f"{path}/snapshot_end.json")
    snapshot_end = json.load(file)

    # load message list
    file = open(f"{path}/message_list.json")
    message_list = json.load(file)
    end = timer()
    print('loading time: ', round(end-start, 2), 'seconds')
    return snapshot_start, snapshot_end, message_list

In [7]:
snapshot_start, snapshot_end, message_list = load_data(path=path+file)
message_list = message_list[1:]
snapshot_start = snapshot_start[0]

NameError: name 'timer' is not defined

In [8]:
market_state = MarketStateAttribute(market_id="0", l3_levels=7, l2_levels=8, report_state_timestamps=True)

In [9]:
market_state.instance.initialize_state(snapshot_start)

(INFO) State Build from Snapshot: Start Timestamp: 2022-02-02 08:00:11.923329580


In [10]:
for idx, message_packet in enumerate(message_list):
    market_state.update_with_exchange_message(message_packet)

Flawed Message: 
{'MessageHeader': {'BodyLen': 80, 'TemplateID': 13202, 'MsgSeqNum': 106970}, 'SecurityID': '2504900', 'AggressorTime': None, 'RequestTime': None, 'ExecID': '1643791765054516679', 'LastQty': '150000', 'AggressorSide': 2, 'TradeCondition': None, 'LastPx': '9400000000', 'RestingHiddenQty': '0', 'RestingCxlQty': '0'}
None


### Validate Internal State with Snapshot End

 -> if the final internal state is equal to the final snapshot, the reconstruction of the order book
 from the message stream was successful

In [11]:
# parse snapshot end
market_state.validate_state(snapshot_end)

True

### Further Analysis

In [184]:
agent_message_list = [{'template_id': 99999, 'side': 1, 'price': 9411000000, 'quantity': 2220000, 'timestamp': 1643716931606681391}, 
                      {'template_id': 99999, 'side': 1, 'price': 9413000000, 'quantity': 1110000, 'timestamp': 1643716931606681391}, 
                      {'template_id': 99999, 'side': 2, 'price': 9436000000, 'quantity': 3330000, 'timestamp': 1643716931606681391}, 
                      {'template_id': 99999, 'side': 2, 'price': 9438000000, 'quantity': 8880000, 'timestamp': 1643716931606681391}, 
                      {'template_id': 66666, 'side': 2, 'price': 9428000000, 'timestamp': 1643716931606681391}]

In [185]:
buy_prices = []
sell_prices = []
for message in agent_message_list:
    # filter for order submissions (exclude cancellations):
    if message['template_id'] == 99999:
        if message['side'] == 1:
            buy_prices.append(message['price'])
        if message['side'] == 2:
            sell_prices.append(message['price'])

max_buy_order_price = max(buy_prices)
min_buy_order_price = min(buy_prices)
max_sell_order_price = max(sell_prices)
min_sell_order_price = min(sell_prices)

In [186]:
print('max_buy_order_price',max_buy_order_price)
print('min_buy_order_price', min_buy_order_price)
print('max_sell_order_price', max_sell_order_price)
print('min_sell_order_price', min_sell_order_price)

max_buy_order_price 9413000000
min_buy_order_price 9411000000
max_sell_order_price 9438000000
min_sell_order_price 9436000000


In [187]:
buy_prices

[9411000000, 9413000000]

In [188]:
sell_prices

[9436000000, 9438000000]

In [189]:
# Simulation State Dimension:

# For Agent Buy Order:
# - Ask-State: ask_level =< min_buy_order_price (no matching possible if ask > bid)
# - Bid-State: bid-level => max_buy_order_price (lob buy orders with same or higher bid limit could have higher priority)

# For Agent Sell Order:
# Bid-State: bid-level => min_sell_order_price (no matching if bid < ask)
# Ask-Side: ask-level =< max_sell_order_price (lob sell orders with same or lower ask limit could have higher priority)

# Hence:

# Ask Thresh: ask_level =< min_buy_order_price AND ask-level =< max_sell_order_price (SMALLER CONDITION)

# Bid Thresh: bid-level => max_buy_order_price AND bid-level => min_sell_order_price (LARGER CONDITION)

# simulation_state dimension:
# - all bid-levels => bid_threshold
# - all ask-levels =< ask_threshold

In [190]:
#TODO: MAX OR MIN???
ask_threshold = max(min_buy_order_price, max_sell_order_price)
bid_threshold = min(max_buy_order_price, min_sell_order_price)

In [191]:
print('ask_threshold', ask_threshold)
print('bid_threshold', bid_threshold)

ask_threshold 9438000000
bid_threshold 9413000000


In [192]:
_state = copy.deepcopy(market_state._state)

In [193]:
# Bid-side-keys
bid_keys = list(_state[1].keys())
bid_keys_relevant = list(i for i in bid_keys if i >= bid_threshold)
bid_keys_relevant

[9413000000,
 9414000000,
 9415000000,
 9416000000,
 9417000000,
 9418000000,
 9419000000,
 9420000000,
 9421000000,
 9422000000,
 9423000000,
 9424000000,
 9425000000,
 9426000000,
 9427000000,
 9428000000,
 9429000000,
 9431000000,
 9430000000,
 9432000000]

In [194]:
# Bid-side-keys
ask_keys = list(_state[2].keys())
ask_keys_relevant = list(i for i in ask_keys if i <= ask_threshold)
ask_keys_relevant

[9438000000, 9437000000, 9436000000, 9435000000, 9434000000]

In [195]:
simulation_state = {}
bid_side_dict = {}
ask_side_dict = {}

# store relevant bid levels to bid_side_dict
for key in bid_keys_relevant:
    if key in _state[1]:
        bid_side_dict[key] = _state[1][key]
        
# store relevant ask levels to ask_side_dict
for key in ask_keys_relevant:
    if key in _state[2]:
        ask_side_dict[key] = _state[2][key]

# store relevant levels to simulation_state
simulation_state[1] = bid_side_dict
simulation_state[2] = ask_side_dict

In [196]:
print(len(bid_keys_relevant))
print(len(simulation_state[1]))

20
20


In [197]:
simulation_state[1].keys()

dict_keys([9413000000, 9414000000, 9415000000, 9416000000, 9417000000, 9418000000, 9419000000, 9420000000, 9421000000, 9422000000, 9423000000, 9424000000, 9425000000, 9426000000, 9427000000, 9428000000, 9429000000, 9431000000, 9430000000, 9432000000])

In [198]:
simulation_state[2].keys()

dict_keys([9438000000, 9437000000, 9436000000, 9435000000, 9434000000])

#### Append Agent messages to simulation state

In [199]:
# add agent messages to simulation_state:
for message in agent_message_list:
    print(message)
    if message['template_id'] == 99999:
        # buy-order
        if message['side'] == 1:
            price = message['price']
            # add message if price level exists
            if price in simulation_state[1]:
                simulation_state[1][price].append(message)
            # create new price level for message
            else:
                simulation_state[1][price] = [message]
        # sell-order
        if message['side'] == 2:
            price = message['price']
            if price in simulation_state[2]:
                simulation_state[2][price].append(message)
            else:
                simulation_state[2][price] = [message]

{'template_id': 99999, 'side': 1, 'price': 9411000000, 'quantity': 2220000, 'timestamp': 1643716931606681391}
{'template_id': 99999, 'side': 1, 'price': 9413000000, 'quantity': 1110000, 'timestamp': 1643716931606681391}
{'template_id': 99999, 'side': 2, 'price': 9436000000, 'quantity': 3330000, 'timestamp': 1643716931606681391}
{'template_id': 99999, 'side': 2, 'price': 9438000000, 'quantity': 8880000, 'timestamp': 1643716931606681391}
{'template_id': 66666, 'side': 2, 'price': 9428000000, 'timestamp': 1643716931606681391}


In [200]:
# Bid
sorted(list(simulation_state[1].keys()))

[9411000000,
 9413000000,
 9414000000,
 9415000000,
 9416000000,
 9417000000,
 9418000000,
 9419000000,
 9420000000,
 9421000000,
 9422000000,
 9423000000,
 9424000000,
 9425000000,
 9426000000,
 9427000000,
 9428000000,
 9429000000,
 9430000000,
 9431000000,
 9432000000]

In [201]:
# ask
list(simulation_state[2].keys())

[9438000000, 9437000000, 9436000000, 9435000000, 9434000000]

In [202]:
market_state.state_l1

{0: 1643803200024859829, 1: {9432000000: 2450000}, 2: {9434000000: 1860000}}

In [204]:
agent_message_list[0]

{'template_id': 99999,
 'side': 1,
 'price': 9411000000,
 'quantity': 2220000,
 'timestamp': 1643716931606681391}

In [209]:
a = 1
b = 2

if a == 1 and b == 2:
    print('true')

true


In [211]:
market_state.state_l1[1]

{9432000000: 2450000}

In [212]:
if market_state.state_l1[1]:
    print('gut')

gut


In [215]:
test = {0: 1643803200024859829, 1: {2}, 2: {9434000000: 1860000}}

In [216]:
if test[1]:
    print('gut')

gut


#### use trade-list to update agent_message_list

In [ ]:
#0) Problem: my executed orders could be either aggressors or liquidity providers... the trade_list only
# contains agressors so far...


# 1) Identify Order
# 2) determine if fully or partially executed
# 3) if fully executed, change order template_id to 11111
# 4) If partially executed...? i have to change quantity, cannot just change template_id...

In [217]:
trade_list = [{'aggressor_side': 1, 'price': 9418000000, 'timestamp': 1643716931606681391, 'quantity': 250000}]

In [218]:
agent_message_list

[{'template_id': 99999,
  'side': 1,
  'price': 9411000000,
  'quantity': 2220000,
  'timestamp': 1643716931606681391},
 {'template_id': 99999,
  'side': 1,
  'price': 9413000000,
  'quantity': 1110000,
  'timestamp': 1643716931606681391},
 {'template_id': 99999,
  'side': 2,
  'price': 9436000000,
  'quantity': 3330000,
  'timestamp': 1643716931606681391},
 {'template_id': 99999,
  'side': 2,
  'price': 9438000000,
  'quantity': 8880000,
  'timestamp': 1643716931606681391},
 {'template_id': 66666,
  'side': 2,
  'price': 9428000000,
  'timestamp': 1643716931606681391}]

### Einfacher: Kompett neue Matching Logik

In [249]:
state_to_match = copy.deepcopy(market_state._state)

In [256]:
# insert executable order for testing
state_to_match[1][9463000000] = [{'template_id': 13100,
    'msg_seq_num': 330218,
    'side': 1,
    'price': 9463000000,
    'quantity': 888888,
    'timestamp': 1643800110844504171,
    'time-in': 1643800110844495601}]

In [227]:
state_to_match[1][9463000000]

[{'template_id': 13100,
  'msg_seq_num': 330218,
  'side': 1,
  'price': 9463000000,
  'quantity': 888888,
  'timestamp': 1643800110844504171,
  'time-in': 1643800110844495601}]

In [242]:
# TODO: determine the right execution price!
# Es sollte immer der Preis sein der länger im Buch steht (siehe EFN1)

In [244]:

trade_list = []

while True:
    max_buy = max(state_to_match[1].keys())  # best bid
    min_sell = min(state_to_match[2].keys())  # best ask
    print(max_buy, min_sell)
    
    # check if execution is possible
    if not max_buy >= min_sell:
        break
        
    # if empty, remove the current best price_level and continue with the loop
    if not (price_level := state_to_match[1][max_buy]): # :=  allows variable assignment inside expression
        del state_to_match[1][max_buy];
        continue
        
    if not (price_level := state_to_match[2][min_sell]):
        del state_to_match[2][min_sell];
        continue
    
    # TODO: Orders müssen erst nach time priority sortiert werden! (zumindest in simulation)
    
    # in the following, match these two particular orders
    # sort by priority time and select first order (lowest priority time, necessary for simulation!)
    order_buy = sorted(state_to_match[1][max_buy], key=lambda d: d['timestamp'])[0]
    order_sell = sorted(state_to_match[2][min_sell], key=lambda d: d['timestamp'])[0]
    
    
    # assign roles for these two particular orders
    order_aggressor, order_standing = sorted([order_buy, order_sell],
                                             key=lambda x: x["timestamp"], reverse=True)
    
    # check quantity delta
    delta = order_buy["quantity"] - order_sell["quantity"]
    
    # (buy - sell) < 0: buy executed at min_sell, sell reduced to delta
    
    ###### DELTA < 0 ---> price = order_sell (?) ######
    
    # Error Szenario:
    #
    # - Limit Buy Order sits in the book (order_standing); quantity = 50
    # - Limit Sell Order arrives (aggressor); quantity = 100
    # => Price = Highest Bid Limit
    #
    # delta = -50
    # => Used Execution Price = Sell Order Limit
    #
    # Problem: Highest Bid Limit could be smaller than Sell Order Limit
    
    if delta < 0:
        # remove qt from partially executed sell order
        order_sell["quantity"] -= order_buy["quantity"]
        # store to trade-list
        trade_list.append({"aggressor_side": order_aggressor["side"],
                           "price": order_sell["price"],  # min_sell
                           "timestamp": order_aggressor["timestamp"],
                           "quantity": order_buy["quantity"],
                           })
        
        # remove the fully executed buy order from state
        state_to_match[1][max_buy].remove(order_buy)
    
    # (buy - sell) > 0: sell executed at max_buy, buy reduced to delta
    
    ###### DELTA > 0 ---> price = order_buy (?) ######
    
    elif delta > 0:
        # remove qt from partially executed buy order
        order_buy["quantity"] -= order_sell["quantity"]
        # store to trade-list
        trade_list.append({"aggressor_side": order_aggressor["side"],
                           "price": order_buy["price"],  # max_buy
                           "timestamp": order_aggressor["timestamp"],
                           "quantity": order_sell["quantity"],
                           })
        # remove executed sell order from state
        state_to_match[2][min_sell].remove(order_sell)
        
    # (buy - sell) > 0: sell executed at max_buy, buy reduced to delta
    
    ###### DELTA = 0 ---> price = order_standing (!) ######
    
    else:
        trade_list.append({"aggressor_side": order_aggressor["side"],
                           "price": order_standing["price"],
                           "timestamp": order_aggressor["timestamp"],
                           "quantity": order_buy["quantity"],  # = order_sell["quantity"]
                           })
        
        # remove both fully executed orders from state
        state_to_match[1][max_buy].remove(order_buy)
        state_to_match[2][min_sell].remove(order_sell)
    
    #self._update_internal_timestamp()
    
    #return trade_list
        
        
    

9463000000 9434000000
9463000000 9434000000
9463000000 9434000000
9432000000 9434000000


In [ ]:
# Matching Logic:

# 2 Scenarios:
# -----------

# 1) 
# - Limit Buy Order sits in the book (order_standing)
# - Limit Sell Order arrives (aggressor)
# => Price = Highest Bid Limit (i.e. limit of buy order standing)

# 2)
# - Limit Sell Order sits in the book (order_standing)
# - Limit Buy Order arrives (aggressor)
# => Price = Lowest Ask Limit (i.e. limit of sell order standing)
#
# Igore Edge Cases (only market orders in the book etc.)
#
# ==> The price of the order standing can always be used as Execution Price!




In [254]:
# -- V2 (08/Sept/2022)
trade_list = []

# iterate until break
while True:
    
    max_buy = max(state_to_match[1].keys())  # best bid
    min_sell = min(state_to_match[2].keys())  # best ask
    #DEBUGGING
    print('max_buy: ', max_buy, 'min_sell: ', min_sell)
    
    # break if order book not crossed
    if not max_buy >= min_sell:
        break
        
    # if empty, remove the current best price_level and continue with the loop
    if not (price_level := state_to_match[1][max_buy]): # :=  allows variable assignment inside expression
        del state_to_match[1][max_buy];
        continue
        
    if not (price_level := state_to_match[2][min_sell]):
        del state_to_match[2][min_sell];
        continue
    
    # in the following, match these two particular orders:
    
    # sort by priority time and select first order (lowest priority time, necessary for simulation!)
    order_buy = sorted(state_to_match[1][max_buy], key=lambda d: d['timestamp'])[0]
    order_sell = sorted(state_to_match[2][min_sell], key=lambda d: d['timestamp'])[0]
    
    # aggressor order has later timestamp
    order_standing, order_aggressor = sorted([order_buy, order_sell],key=lambda x: x["timestamp"])
    
    # execution price is always the price of the standing order
    execution_price = order_standing['price']
    aggressor_side = order_aggressor["side"]
    aggressor_timestamp = order_aggressor["timestamp"]
    
    # check quantity delta
    delta = order_buy["quantity"] - order_sell["quantity"]
    order_aggressor["side"]

    # TODO: Also, remove quantity from part-matched orders in agent_trade_list...
    
    if delta < 0:
        # remove qt from partially executed sell order
        order_sell["quantity"] -= order_buy["quantity"]
        # save executed quantity for trade report
        execution_quantity = order_buy["quantity"]
        # remove the fully executed buy order from state
        state_to_match[1][max_buy].remove(order_buy)
    
    elif delta > 0:
        # remove qt from partially executed buy order
        order_buy["quantity"] -= order_sell["quantity"]
        # store to trade-list
        execution_quantity = order_sell["quantity"]
        # remove executed sell order from state
        state_to_match[2][min_sell].remove(order_sell)
        
    
    else:
        # both orders fully executed
        execution_quantity = order_sell["quantity"]
        # remove both fully executed orders from state
        state_to_match[1][max_buy].remove(order_buy)
        state_to_match[2][min_sell].remove(order_sell)
        
    
    # report execution in trade_list
    trade_list.append({"aggressor_side": aggressor_side,
                        "price": execution_price,
                        "timestamp": aggressor_timestamp,
                        "quantity": execution_quantity,
                    })
    
    #self._update_internal_timestamp()
    
    #DEBUGGING
    print(trade_list)
    #return trade_list

max_buy:  9463000000 min_sell:  9434000000
order_standing {'template_id': 13100, 'msg_seq_num': 330218, 'side': 1, 'price': 9463000000, 'quantity': 888888, 'timestamp': 1643800110844504171, 'time-in': 1643800110844495601}
order_aggressor {'template_id': 13100, 'msg_seq_num': 395931, 'side': 2, 'price': 9434000000, 'quantity': 241112, 'timestamp': 1643803200016020618, 'time-in': 1643803200016011415}
[{'aggressor_side': 2, 'price': 9463000000, 'timestamp': 1643803200016020618, 'quantity': 241112}]
max_buy:  9463000000 min_sell:  9434000000
order_standing {'template_id': 13100, 'msg_seq_num': 330218, 'side': 1, 'price': 9463000000, 'quantity': 647776, 'timestamp': 1643800110844504171, 'time-in': 1643800110844495601}
order_aggressor {'template_id': 13100, 'msg_seq_num': 395932, 'side': 2, 'price': 9434000000, 'quantity': 730000, 'timestamp': 1643803200016027679, 'time-in': 1643803200016011639}
[{'aggressor_side': 2, 'price': 9463000000, 'timestamp': 1643803200016020618, 'quantity': 241112}

In [263]:
bid_prices = [1,2,16, 3,4,5,6,7,8,9,10]

In [261]:
for price in bid_prices:

    if price >= 4:
        outer_bid_price = price
        break

In [262]:
outer_bid_price

4

In [269]:
a = bid_prices.sort(reverse=True)

In [268]:
a

In [270]:
bid_prices

[16, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]